Description

Embedding Model : text-embedding-3-small , sentence-transformers/LaBSE

Vector DB : MongoDB

โดยการทดสอบมีวิธีการที่แตกต่างกันสำหรับการทำ RAG และ การทำ Tokenize


In [18]:
import os
import pandas as pd
import time
import nest_asyncio
import asyncio
import numpy as np
from openai import AsyncOpenAI
from pymongo import MongoClient
import fitz  # PyMuPDF
from docx import Document
import tiktoken
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

load_dotenv()

nest_asyncio.apply()

In [2]:
#KEY 
OPENAI_API_KEY : str 
EMBEDDING : str 
MONGO_URI : str

MONGO_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING = os.getenv("EMBEDDING")
client_openai = AsyncOpenAI(api_key=OPENAI_API_KEY)
mongo_client = MongoClient(MONGO_URI)
db = mongo_client["vector_db"]
collection = db["vectors"]

In [ ]:
file_path = "../data"
# 3. Define helper functions
def read_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def read_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

async def embed_batch(batch, embed_model):
    response = await client_openai.embeddings.create(model=embed_model, input=batch)
    return [item.embedding for item in response.data]

async def batch_process_embedding_async(text_list, embed_model, batch_size=100):
    tasks = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        tasks.append(embed_batch(batch, embed_model))
    results = await asyncio.gather(*tasks)
    embeddings = [embedding for batch in results for embedding in batch]
    return embeddings

def cosine_similarity(vec1, vec2):
    vec1, vec2 = np.array(vec1), np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def count_tokens(text, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# ฟังก์ชันนับจำนวนโทเค็น
def count_tokens_2(text, model="sentence-transformers/LaBSE"):
    # โหลด tokenizer สำหรับโมเดลที่กำหนด
    tokenizer = AutoTokenizer.from_pretrained(model)
    # ใช้ tokenizer ในการแปลงข้อความเป็นโทเค็นและนับจำนวนโทเค็น
    encoded = tokenizer.encode(text, truncation=True, padding=False)
    return len(encoded)

In [4]:
start_upload = time.perf_counter()

dfs = []

for filename in os.listdir(file_path):
    full_path = os.path.join(file_path, filename)
    if filename.endswith('.csv'):
        df = pd.read_csv(full_path)
        dfs.append(df)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(full_path, sheet_name=None)
        for sheet in df.values():
            dfs.append(sheet)
    elif filename.endswith('.docx'):
        text = read_docx(full_path)
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)
    elif filename.endswith('.pdf'):
        text = read_pdf(full_path)
        df = pd.DataFrame({"text": [text]})
        dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)
df_combined.dropna(inplace=True)
df_combined.drop_duplicates(inplace=True)
df_combined.reset_index(drop=True, inplace=True)

end_upload = time.perf_counter()
print(f"✅ Upload เสร็จ {len(df_combined)} records ในเวลา {end_upload - start_upload:.2f} วินาที")
print("✅ Shape after cleansing:", df_combined.shape)

✅ Upload เสร็จ 250 records ในเวลา 0.02 วินาที
✅ Shape after cleansing: (250, 11)


Pattern 1

In [5]:
texts = []
metadata_list = []
for i, row in df_combined.iterrows():
    metadata = row.to_dict()
    text = "\n".join([f"{k}: {v}" for k, v in metadata.items()])
    texts.append(text)
    metadata_list.append((f"vec-{i}", metadata))


In [6]:
start_embed = time.perf_counter()

embeddings = await batch_process_embedding_async(texts, EMBEDDING)

end_embed = time.perf_counter()
embed_time = end_embed - start_embed
print(f"✅ Embedding เสร็จ {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")

✅ Embedding เสร็จ 250 records ในเวลา 3.40 วินาที


In [7]:
start_upsert = time.perf_counter()

collection.delete_many({})  
documents = []
for (vec_id, metadata), embedding, raw_text in zip(metadata_list, embeddings, texts):
    documents.append({
        "_id": vec_id,
        "embedding": embedding,
        "metadata": metadata,
        "raw_text": raw_text
    })

collection.insert_many(documents)
print(f"✅ Inserted {len(documents)} documents into MongoDB.")

end_upsert = time.perf_counter()
upsert_time = end_upsert - start_upsert
print(f"✅ Upsertเสร็จ {len(documents)} records ในเวลา {upsert_time :.2f} วินาที")


✅ Inserted 250 documents into MongoDB.
✅ Upsertเสร็จ 250 records ในเวลา 0.17 วินาที


In [8]:
async def retrieve_context_from_mongodb(question: str, top_k: int = 50):
    embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    question_vector = embedder.embed_query(question)

    documents = list(collection.find())
    similarities = []
    for doc in documents:
        score = cosine_similarity(question_vector, doc["embedding"])
        similarities.append((score, doc))

    similarities.sort(reverse=True, key=lambda x: x[0])
    top_docs = [doc["raw_text"] for score, doc in similarities[:top_k]]
    return "\n".join(top_docs)

In [9]:
start_q = time.perf_counter()

question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_mongodb(question)
num_tokens = count_tokens(context)
print(f"จำนวน token ทั้งหมดใน context: {num_tokens}")

จำนวน token ทั้งหมดใน context: 3140


In [10]:
prompt = f"""
ข้อมูลต่อไปนี้มาจากไฟล์ในรูปแบบตาราง เช่น CSV หรือ Excel ซึ่งอาจมีหลายประเภทข้อมูลและหลายแถว:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- วิเคราะห์ข้อมูลทั้งหมดให้ครบถ้วน
- สรุปคำตอบโดยใช้ภาษาธรรมดา ชัดเจน และไม่ใช้การจัดรูปแบบตัวหนา หัวข้อ หรือสัญลักษณ์พิเศษ เช่น ** หรือ -
- หากมีหลายรายการสินค้า ให้ระบุชื่อสินค้า พร้อมจำนวน และหน่วยตามลักษณะของสินค้า เช่น เครื่อง, ตัว, คู่, เล่ม ฯลฯ
- หากต้องรวมจำนวน ให้รวมและแสดงยอดรวมทั้งหมด
- ตอบเป็นภาษาไทยแบบเป็นธรรมชาติ เข้าใจง่าย เหมือนอธิบายให้คนทั่วไปฟัง
- หากไม่พบข้อมูลที่เกี่ยวข้อง ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง"
"""

llm = ChatOpenAI(temperature=0, model="gpt-4", api_key=OPENAI_API_KEY)
response = llm.invoke(prompt)

end_q = time.perf_counter()
response_time = end_q - start_q
print("คำตอบ:", response.content)
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")

คำตอบ: จากการวิเคราะห์ข้อมูล พบว่ามีสินค้าทั้งหมด 7 ประเภท ดังนี้:

1. T-Shirt จำนวน 3 ตัว
2. Laptop จำนวน 15 เครื่อง
3. Smartphone จำนวน 4 เครื่อง
4. Jeans จำนวน 4 ตัว
5. Refrigerator จำนวน 3 เครื่อง
6. Washing Machine จำนวน 3 เครื่อง
7. Smartwatch จำนวน 4 เครื่อง
8. Running Shoes จำนวน 4 คู่

ดังนั้น สรุปได้ว่ามีสินค้าทั้งหมด 7 ประเภท และมีจำนวนรวมทั้งสิ้น 40 รายการ
⏱ ใช้เวลาในการตอบ: 12.58 วินาที


In [11]:
print(f"✅ Embedding เสร็จทั้งหมด {len(embeddings)} records ในเวลา {embed_time:.2f} วินาที")
print(f"✅ Upsert MongoDB เสร็จ {len(documents)} vectors ในเวลา {upsert_time :.2f} วินาที")
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")
print(f"เวลาโดยรวมupload MongoDB ทั้งหมด : {embed_time+upsert_time:.2f} ")

✅ Embedding เสร็จทั้งหมด 250 records ในเวลา 3.40 วินาที
✅ Upsert MongoDB เสร็จ 250 vectors ในเวลา 0.17 วินาที
⏱ ใช้เวลาในการตอบ: 12.58 วินาที
เวลาโดยรวมupload MongoDB ทั้งหมด : 3.57 


Pattern 2

In [ ]:
# โหลด tokenizer จาก BERT
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# โหลดโมเดล LaBSE สำหรับการสร้าง embeddings
labse_model = SentenceTransformer('sentence-transformers/LaBSE')

def reduce_vector_dimension(vec, target_dim):
    pca = PCA(n_components=target_dim)
    return pca.fit_transform(vec.reshape(1, -1)).flatten()

# ฟังก์ชัน cosine similarity ที่รองรับเวกเตอร์ 1D
def cosine_similarity_2(vec1, vec2):
    vec1 = np.array(vec1).flatten()
    vec2 = np.array(vec2).flatten()
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0  # ป้องกันหารด้วยศูนย์
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# ฟังก์ชันลดจำนวน token โดยใช้ tokenizer ของ BERT
def reduce_token_with_bert(text, max_tokens=300):
    encoded = bert_tokenizer(text, truncation=True, max_length=max_tokens, return_tensors='pt')
    # แปลง token กลับเป็นข้อความหลังตัด token แล้ว
    truncated_text = bert_tokenizer.decode(encoded["input_ids"][0], skip_special_tokens=True)
    return truncated_text

# ฟังก์ชัน cosine similarity ที่รองรับเวกเตอร์ 1D
def cosine_similarity_2(vec1, vec2):
    vec1 = np.array(vec1).flatten()
    vec2 = np.array(vec2).flatten()
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0  # ป้องกันหารด้วยศูนย์
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
async def retrieve_context_from_mongodb(question: str, top_k: int = 50):
    # สร้าง embedding สำหรับคำถามด้วย LaBSE
    question_vector = labse_model.encode([question], convert_to_numpy=True)

    # ดึงข้อมูลจาก MongoDB
    documents = list(collection.find())
    similarities = []
    
    # คำนวณ cosine similarity สำหรับแต่ละเอกสาร
    for doc in documents:
        doc_embedding = np.array(doc["embedding"]).flatten()

        # ปรับขนาดเวกเตอร์ของ doc["embedding"] ให้ตรงกับขนาดของ question_vector
        target_dim = question_vector.shape[0]  # ใช้ขนาดของ question_vector
        if doc_embedding.shape[0] != target_dim:
            doc_embedding = reduce_vector_dimension(doc_embedding, target_dim)

        # คำนวณ cosine similarity
        score = cosine_similarity_2(question_vector, doc_embedding)
        similarities.append((score, doc))

    # จัดเรียงตามคะแนน similarity
    similarities.sort(reverse=True, key=lambda x: x[0])

    # คืนค่าข้อความที่ถูกลดจำนวนโทเค็น
    reduced_texts = []
    for score, doc in similarities[:top_k]:
        reduced = reduce_token_with_bert(doc["raw_text"], max_tokens=300)
        reduced_texts.append(reduced)

    return "\n".join(reduced_texts)

In [ ]:


question = "สินค้ามีอะไรบ้าง และมีจำนวนเท่าไร"
context = await retrieve_context_from_mongodb(question)
num_tokens_context = count_tokens(context)
print(f"จำนวนโทเค็นใน context: {num_tokens_context}")

c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\user\OneDrive\Desktop\Test_BOT_CSV\venv\Lib\site-packages\sklearn\decomposition\_pca.py:586: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (

จำนวนโทเค็นใน context: 512


In [ ]:
num_tokens_context = count_tokens_2(context)
print(f"จำนวนโทเค็นใน context: {num_tokens_context}")

จำนวนโทเค็นใน context: 512


In [17]:
# สร้าง prompt
start_q = time.perf_counter()
prompt = f"""
ข้อมูลต่อไปนี้มาจากไฟล์ในรูปแบบตาราง เช่น CSV หรือ Excel ซึ่งอาจมีหลายประเภทข้อมูลและหลายแถว:
{context}

คำถามของฉันคือ: "{question}"

กรุณาตอบโดย:
- วิเคราะห์ข้อมูลทั้งหมดให้ครบถ้วน
- สรุปคำตอบโดยใช้ภาษาธรรมดา ชัดเจน และไม่ใช้การจัดรูปแบบตัวหนา หัวข้อ หรือสัญลักษณ์พิเศษ เช่น ** หรือ - 
- หากมีหลายรายการสินค้า ให้ระบุชื่อสินค้า พร้อมจำนวน และหน่วยตามลักษณะของสินค้า เช่น เครื่อง, ตัว, คู่, เล่ม ฯลฯ
- หากต้องรวมจำนวน ให้รวมและแสดงยอดรวมทั้งหมด
- ตอบเป็นภาษาไทยแบบเป็นธรรมชาติ เข้าใจง่าย เหมือนอธิบายให้คนทั่วไปฟัง
- หากไม่พบข้อมูลที่เกี่ยวข้อง ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง"
"""

llm = ChatOpenAI(temperature=0, model="gpt-4", api_key=OPENAI_API_KEY)
response = llm.invoke(prompt)

# วัดเวลาในการประมวลผล
end_q = time.perf_counter()
response_time = end_q - start_q
print("คำตอบ:", response.content)
print(f"⏱ ใช้เวลาในการตอบ: {response_time:.2f} วินาที")

คำตอบ: จากการวิเคราะห์ข้อมูล พบว่ามีสินค้าทั้งหมด 8 ประเภท ดังนี้

1. รองเท้าวิ่ง (Running Shoes) จำนวน 10 คู่
2. หูฟัง (Headphones) จำนวน 12 ตัว
3. นาฬิกาอัจฉริยะ (Smartwatch) จำนวน 9 ตัว
4. เสื้อยืด (T - Shirt) จำนวน 13 ตัว
5. สมาร์ทโฟน (Smartphone) จำนวน 8 ตัว
6. หนังสือ (Book) จำนวน 9 เล่ม
7. ยีนส์ (Jeans) จำนวน 4 ตัว
8. แล็ปท็อป (Laptop) จำนวน 7 เครื่อง

นอกจากนี้ยังมีสินค้าประเภทอุปกรณ์ในบ้าน 2 ประเภท คือ เครื่องซักผ้า (Washing Machine) จำนวน 5 เครื่อง และ ตู้เย็น (Refrigerator) จำนวน 7 เครื่อง

ดังนั้น สรุปได้ว่ามีสินค้าทั้งหมด 10 ประเภท และจำนวนรวมทั้งหมด 84 รายการ
⏱ ใช้เวลาในการตอบ: 18.90 วินาที
